In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get

# Pulling all links off of the search pages (up to 20) and putting them in a dataframe to scrape.
Will use 20 for the sake of trying it out...change it later.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Data+Scientist%22&l=USA&start={}"
max_results=20
Linkdf=[]

for start in range(0, max_results, 10):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

18

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])
print(websites)

['https://www.indeed.com/rc/clk?jk=267e7cf7de2959c7&fccid=de71a49b535e21cb&vjs=3', 'https://www.indeed.com/rc/clk?jk=1942a17d2bd166e1&fccid=ae0c894528aa6eee&vjs=3', 'https://www.indeed.com/rc/clk?jk=1e686166d30fd118&fccid=de71a49b535e21cb&vjs=3', 'https://www.indeed.com/rc/clk?jk=020c5d3fcc757260&fccid=79462bd4e96de970&vjs=3', 'https://www.indeed.com/rc/clk?jk=12e3127bb1fdf9e9&fccid=de71a49b535e21cb&vjs=3', 'https://www.indeed.com/rc/clk?jk=95f7f6a9d224911d&fccid=3c371b712ace44cf&vjs=3', 'https://www.indeed.com/rc/clk?jk=11baba484db23433&fccid=734cb5a01ee60f80&vjs=3', 'https://www.indeed.com/rc/clk?jk=3d46376452926cef&fccid=a168335bbdcce5e0&vjs=3', 'https://www.indeed.com/rc/clk?jk=3afa4b5a60bab58f&fccid=1639254ea84748b5&vjs=3', 'https://www.indeed.com/rc/clk?jk=63f1c9af7e695c9d&fccid=86b39e8431ac7b5f&vjs=3', 'https://www.indeed.com/rc/clk?jk=b34757e702ad2ee0&fccid=245cecf5f0560d8c&vjs=3', 'https://www.indeed.com/rc/clk?jk=1eb7e96ae99c871d&fccid=1639254ea84748b5&vjs=3', 'https://www.in

# Looping through websites....

In [5]:
#Loop

Title=[]
Descriptions=[]
Location=[]

for url in websites:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    locations=str(location_containers.text)
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)

# iT WORKS!!

In [10]:
#THIS GIVES US WHAT WE NEED BUT PUTS IT ALL IN THE SAME COLUMN
#Search_Values = ['Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement']
#i=0
#
#Qualifications = []

#for n in range(0, len(Descriptions)):
#    for values in Search_Values :
#        soup=BeautifulSoup(Descriptions[n],'lxml')

#        Qualifications_tag = soup.find('b', text=re.compile(values))
#        try:
#            ul_tag = Qualifications_tag.find_next('ul')
#            Qualification=ul_tag.text
#            Qualifications.append({i,Qualification})
#        except AttributeError: 
#            Qualification="None Found"
#            Qualifications.append({i,Qualification})
#    i=i+1

Descriptions_df = pd.DataFrame(columns = ['Title', 'Location', 'Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement'])
#Descriptions_df

for index, element in enumerate(Descriptions):
    soup=BeautifulSoup(element,'lxml')
    for values in list(Descriptions_df):
        temp_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = temp_tag.find_next('ul')
            Descriptions_df.at[index,values] = ul_tag.text
        except AttributeError:
            Descriptions_df.at[index,values]="None Found"
        Descriptions_df.at[index,"Title"]=Title[index]
        Descriptions_df.at[index,"Location"]=Location[index]
        Descriptions_df.at[index,"FullDescription"]=Descriptions[index]

In [11]:
Descriptions_df

Title  \
0   Entry Level Data Scientist                                                          
1   Jr. Data Scientist                                                                  
2   Entry Level - Associate Data Scientist                                              
3   Data Scientist Intern, Boston                                                       
4   Intern Conversion - Data Scientist                                                  
5   Data Scientist                                                                      
6   Full Time Opportunities for Students or Recent Graduates: Data & Applied Sciences   
7   Associate Data Scientist (Full Time Starting Summer 2020)                           
8   Data Scientist, Analytics (University Grad)                                         
9   Data Scientist - Modelling 91819                                                    
10  DATA SCIENTIST I                                                                    
11  Data Scientist, Analytics, Intern                                                   
12  Data Scientist                                                                      
13  Data Scientist (Full Time) - United States                                          
14  2020 Intern - Data Scientist                                                        
15  Data Scientist Intern                                                               
16  Data Scientist                                                                      
17  Data Scientist Analyst - 91719                                                      

                Location  \
0   United States          
1   New York, NY 10036     
2   United States          
3   Boston, MA             
4   United States          
5   Englewood, CO 80113    
6   Redmond, WA            
7   Washington, DC 20036   
8   New York, NY 10017     
9   Dearborn, MI           
10  Dallas, TX             
11  New York, NY 10017     
12  New York, NY           
13  San Jose, CA           
14  United States          
15  Broomfield, CO 80021   
16  Glen Mills, PA         
17  Dearborn, MI           

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Qualifications  \
0   None Found                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

# BELOW HERE DOESN'T MATTER. KEEPING FOR THE SAKE OF RECORDS

# THIS WAS THE LAST ONE WE WERE WORKING ON - DOESN'T QUITE DO WHAT WE WANT YET.

In [ ]:
#THIS WAS THE LAST ONE WE WERE WORKING ON - DOESN'T QUITE DO WHAT WE WANT YET.
Search_Values = ['Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement']
i=0

Qualifications = []
Skills = []
Responsibilities = []
Education = []
Requirements = []
Master = []

for n in range(0, len(Descriptions)):
    for values in Search_Values :
        soup=BeautifulSoup(Descriptions[n],'lxml')

        Qualifications_tag = soup.find('b', text=re.compile(values))
        temp=[]
        try:
            ul_tag = Qualifications_tag.find_next('ul')
            Qualification=ul_tag.text
            temp.append({i,Qualification})
            df = pd.DataFrame(temp)
        except AttributeError: 
            Qualification="None Found"
            temp.append({i,Qualification})
            df = pd.DataFrame(temp)
        #Master[m].append(df)
        #m = m +1
   # Master[m].append(Skills)
    i=i+1        

# Below here works, just need to get the above working.

In [ ]:
#Full descriptions in dataframe
df_descriptions = pd.DataFrame({'Full_Description': Descriptions})
df_descriptions

In [ ]:
df_title=pd.DataFrame({'Job_Title':Title})
df_title

In [ ]:
df_locations=pd.DataFrame({'Location':Location})
df_locations

In [ ]:
FullDataSet = pd.concat([df_title,df_locations, df_descriptions], axis=1)
FullDataSet